In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from art.defences.trainer import AdversarialTrainer
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import TensorFlowClassifier
from art.utils import load_mnist

c:\users\좌\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\좌\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\좌\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\좌\appdata\local\progra

In [2]:
# 데이터 셋을 불러온다

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

In [3]:
# 학습을 진행할 모델을 생성한다

input_ph = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
labels_ph = tf.placeholder(tf.int32, shape=[None, 10])

x = tf.layers.conv2d(input_ph, filters=4, kernel_size=5, activation=tf.nn.relu)
x = tf.layers.max_pooling2d(x, 2, 2)
x = tf.layers.conv2d(x, filters=10, kernel_size=5, activation=tf.nn.relu)
x = tf.layers.max_pooling2d(x, 2, 2)
x = tf.layers.flatten(x)
x = tf.layers.dense(x, 100, activation=tf.nn.relu)
logits = tf.layers.dense(x, 10)

loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=labels_ph))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [4]:
# 모든 종류의 attack을 apply 하는 TensorFlow classifer 선언한다

classifier = TensorFlowClassifier(
    clip_values=(min_pixel_value, max_pixel_value),
    input_ph=input_ph,
    output=logits,
    labels_ph=labels_ph,
    train=train,
    loss=loss,
    learning=None,
    sess=sess,
    preprocessing_defences=[],
)

In [5]:
# 위에서 선언한 classifer를 앞서 불러온 데이터 셋을 이용하여 학습시킨다

classifier.fit(x_train, y_train, batch_size=64, nb_epochs=3)

In [6]:
# 공격을 진행하지 않은 데이터를 가지고 위에서 학습시킨 classifer를 이용하여 정확도를 분석한다 

predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("기존의 데이터를 이용한 학습의 정확도 : {}%".format(accuracy * 100))

기존의 데이터를 이용한 학습의 정확도 : 97.68%


In [7]:
# 적대적 공격을 이용하여 기존의 데이터를 공격한다

attack = FastGradientMethod(estimator=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

In [8]:
# 위에서 만들어진 공격된 데이터를 가지고 classifer를 이용하여 정확도를 분석한다

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("적대적 공격에 의해 공격당한 데이터를 이용한 학습의 정확도 : {}%".format(accuracy * 100))

적대적 공격에 의해 공격당한 데이터를 이용한 학습의 정확도 : 29.5%


In [12]:
trainer = AdversarialTrainer(
    classifier,
    attack,
    ratio=1.0
)

In [14]:
trainer.fit(x_train, y_train, batch_size=64, nb_epochs=3)

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
# 위에서 만들어진 공격된 데이터를 가지고 classifer를 이용하여 정확도를 분석한다

predictions = trainer.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("적대적 공격에 의해 공격당한 데이터를 이용한 학습의 정확도 : {}%".format(accuracy * 100))

적대적 공격에 의해 공격당한 데이터를 이용한 학습의 정확도 : 86.08%
